In [7]:
import onnx
import onnx_graphsurgeon as gs
import onnxruntime as ort
import torch
import numpy as np
from rich import print
from onnx import TensorProto

file_name=r"D:\Learn DL\ONNX\bevdet_r50_ckpt_folded_final_folded.onnx"

model=onnx.load(file_name)

In [8]:
session = ort.InferenceSession(model.SerializeToString())
org_outputs = [x.name for x in session.get_outputs()]
org_inputs = [x.name for x in session.get_inputs()]

org_outputs,org_inputs

(['boxes_3d', 'scores_3d', 'labels_3d'],
 ['imgs', 'sensor2keyegos', 'intrins', 'post_rots', 'post_trans', 'bda'])

In [12]:
graph=gs.import_onnx(model)
tensors=graph.tensors()


graph.inputs=[tensors['/img_bev_encoder_backbone/layers.0/layers.0.0/Add_output_0']]
graph.outputs=[tensors['/img_bev_encoder_backbone/layers.0/layers.0.1/Add_output_0']]

graph.cleanup()

sub_model=gs.export_onnx(graph)

onnx.checker.check_model(sub_model)

onnx.save(sub_model, "isolated.onnx")

print("\nONNX model 'isolated.onnx' generated successfully!")

ONNX model 'isolated.onnx' generated successfully!

In [21]:
sub_graph=gs.import_onnx(onnx.load("isolated.onnx"))

relu = [node for node in sub_graph.nodes if node.op=="Relu"][0]

relu

/img_bev_encoder_backbone/layers.0/layers.0.0/relu_1/Relu (Relu)
	Inputs: [
		Variable (/img_bev_encoder_backbone/layers.0/layers.0.0/Add_output_0): (shape=[1, 128, 64, 64], dtype=float32)
	]
	Outputs: [
		Variable (/img_bev_encoder_backbone/layers.0/layers.0.0/relu_1/Relu_output_0): (shape=[1, 128, 64, 64], dtype=float32)
	]

In [ ]:
squeeze_node = gs.Node(op="Squeeze", name="squeeze_node", inputs=[input_tensor], outputs=[squeeze_output], attrs={"axes": [0]})

# Step 3: Create Unsqueeze Node (adds back dimension)
unsqueeze_output = gs.Variable(name="unsqueeze_output", dtype=np.float32, shape=(1, x, x, x))
unsqueeze_node = gs.Node(op="Unsqueeze", name="unsqueeze_node", inputs=[squeeze_output], outputs=[unsqueeze_output], attrs={"axes": [0]})
